In [34]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
from PIL import Image
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Define the paths to your dataset folders
train_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\train_and_validation_sets\train_and_validation_sets"
test_dataset_dir = r"D:\datasets\Underwater_Image\LIMUC (Labeled Images for Ulcerative Colitis)\test_set\test_set"

# Initialize empty lists for X_train, Y_train, X_test, and Y_test
X_train = []
Y_train = []
X_test = []
Y_test = []

# Initialize an empty list to store categorical labels
categorical_labels = []

# Define a function to read and preprocess images
def process_images(folder_path, label, is_train_set=True):
    for filename in os.listdir(folder_path):
        file_path = os.path.join(folder_path, filename)
        if os.path.isfile(file_path) and filename.endswith(".bmp"):  # Check if it's a file and ends with .bmp
            # Open and resize the image to (32, 32, 3)
            img = Image.open(file_path)
            img = img.resize((32, 32))
            img = img.convert("RGB")
            
            # Convert image data to a NumPy array
            img_array = np.array(img).astype('float32')  # Convert to float
            
            # Normalize the image data (optional)
            img_array /= 255.0  # Normalize pixel values to [0, 1]
            
            # Append the image data to the appropriate list
            if is_train_set:
                X_train.append(img_array)
                Y_train.append(label)  # Append the numerical label
            else:
                X_test.append(img_array)
                Y_test.append(label)  # Append the numerical label
            
            # Append the label for categorical encoding
            categorical_labels.append(label)  # Append the numerical label

# List the folders inside the training dataset directory
train_folders = os.listdir(train_dataset_dir)

# Create a label encoder for categorical labels
label_encoder = LabelEncoder()

# Loop through the training folders and process images
for label, folder_name in enumerate(train_folders):
    folder_path = os.path.join(train_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label)

# List the folders inside the test dataset directory
test_folders = os.listdir(test_dataset_dir)

# Loop through the test folders and process images
for label, folder_name in enumerate(test_folders):
    folder_path = os.path.join(test_dataset_dir, folder_name)
    if os.path.isdir(folder_path):  # Check if it's a directory
        process_images(folder_path, label, is_train_set=False)
        
        
        

# Encode Y_train and Y_test categorically
num_classes = len(np.unique(categorical_labels))
  # Update this to match the number of classes

# Convert Y_train and Y_test to NumPy arrays
Y_train = np.array(Y_train)
Y_test = np.array(Y_test)

# Make sure your labels are integers ranging from 0 to num_classes - 1
Y_train = Y_train.astype(int)
Y_test = Y_test.astype(int)

# One-hot encode the labels
Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)



# # Encode Y_train and Y_test categorically using the label encoder
# num_classes = len(np.unique(Y_train))  # Automatically determine the number of classes
# Y_train_categorical = to_categorical(Y_train, num_classes=num_classes)
# Y_test_categorical = to_categorical(Y_test, num_classes=num_classes)


# Check the shape of X_train, Y_train_categorical, X_test, and Y_test_categorical
print("Shape of X_train:", np.shape(X_train))
print("Shape of Y_train_categorical:", np.shape(Y_train_categorical))
print("Shape of X_test:", np.shape(X_test))
print("Shape of Y_test_categorical:", np.shape(Y_test_categorical))

Shape of X_train: (9590, 32, 32, 3)
Shape of Y_train_categorical: (9590, 4)
Shape of X_test: (1686, 32, 32, 3)
Shape of Y_test_categorical: (1686, 4)


In [3]:
train_folders

['train_and_validation_sets']

In [35]:
# Create the ResNet model without the top (fully connected) layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
    
# Add the top layers for classification
model = Sequential()
model.add(base_model)
model.add(GlobalAveragePooling2D())
model.add(Dense(num_classes, activation='softmax'))  # Change units to match the number of classes
model.summary()


Model: "sequential_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 1, 1, 2048)        23587712  
                                                                 
 global_average_pooling2d_11  (None, 2048)             0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_11 (Dense)            (None, 4)                 8196      
                                                                 
Total params: 23,595,908
Trainable params: 23,542,788
Non-trainable params: 53,120
_________________________________________________________________


In [36]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(np.array(X_train).astype('float32'), Y_train_categorical, batch_size=64, epochs=100, validation_split = .2)

Epoch 1/100
120/120 [==============================] - 14s 62ms/step - loss: 0.8970 - accuracy: 0.6856 - val_loss: 77.4025 - val_accuracy: 0.0501
Epoch 2/100
120/120 [==============================] - 7s 58ms/step - loss: 0.5434 - accuracy: 0.7363 - val_loss: 7.1072 - val_accuracy: 0.0000e+00
Epoch 3/100
120/120 [==============================] - 7s 56ms/step - loss: 0.5167 - accuracy: 0.7561 - val_loss: 7.4836 - val_accuracy: 0.0000e+00
Epoch 4/100
120/120 [==============================] - 6s 54ms/step - loss: 0.5515 - accuracy: 0.7407 - val_loss: 14.3168 - val_accuracy: 0.0000e+00
Epoch 5/100
120/120 [==============================] - 7s 57ms/step - loss: 0.5609 - accuracy: 0.7329 - val_loss: 9.1021 - val_accuracy: 5.2138e-04
Epoch 6/100
120/120 [==============================] - 8s 63ms/step - loss: 0.5550 - accuracy: 0.7277 - val_loss: 9.1073 - val_accuracy: 0.0000e+00
Epoch 7/100
120/120 [==============================] - 6s 52ms/step - loss: 0.5719 - accuracy: 0.7323 - val_loss:

120/120 [==============================] - 5s 44ms/step - loss: 0.0527 - accuracy: 0.9814 - val_loss: 15.8379 - val_accuracy: 0.0360
Epoch 58/100
120/120 [==============================] - 6s 50ms/step - loss: 0.0402 - accuracy: 0.9849 - val_loss: 16.1822 - val_accuracy: 0.0282
Epoch 59/100
120/120 [==============================] - 6s 53ms/step - loss: 0.0390 - accuracy: 0.9863 - val_loss: 18.3845 - val_accuracy: 0.0323
Epoch 60/100
120/120 [==============================] - 7s 58ms/step - loss: 0.0432 - accuracy: 0.9838 - val_loss: 20.9187 - val_accuracy: 0.0146
Epoch 61/100
120/120 [==============================] - 6s 51ms/step - loss: 0.0549 - accuracy: 0.9798 - val_loss: 15.2083 - val_accuracy: 0.0276
Epoch 62/100
120/120 [==============================] - 6s 53ms/step - loss: 0.0326 - accuracy: 0.9885 - val_loss: 16.5366 - val_accuracy: 0.0188
Epoch 63/100
120/120 [==============================] - 7s 56ms/step - loss: 0.0327 - accuracy: 0.9892 - val_loss: 14.7296 - val_accuracy

In [37]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")
print("Cohen Kappa Score:")
from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(Y_test, predicted_labels, weights='quadratic')
print(k)

27/27 [==============================] - 2s 23ms/step
Test Loss: 4.727602481842041
Test Accuracy: 0.5824436545372009
Accuracy: 0.5824436536180309
F1 Score: 0.5196370738142837
Balanced Accuracy: 0.31803588070829447
Cohen Kappa Score:
0.2170779668872913


In [38]:
from tensorflow import keras
model = keras.models.load_model(r"C:\Users\shaif\Downloads\LIMUC_32_model.h5")

from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

x = model.layers[-4].output  # Access the last 4th layer from the end
x = GlobalAveragePooling2D()(x)
output = Dense(num_classes, activation='softmax')(x)

# Create the new model with the updated head
new_model = keras.models.Model(inputs=model.input, outputs=output)
# Compile the model
new_model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
# Train the model
new_model.fit(np.array(X_train).astype('float32'), Y_train_categorical, batch_size=64, epochs=100, validation_split = .2)

C:\Users\shaif\anaconda3\envs\tens\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


Epoch 1/100
120/120 [==============================] - 19s 88ms/step - loss: 0.7508 - accuracy: 0.7135 - val_loss: 29.1215 - val_accuracy: 0.0255
Epoch 2/100
120/120 [==============================] - 7s 54ms/step - loss: 0.5469 - accuracy: 0.7449 - val_loss: 18.4478 - val_accuracy: 0.0255
Epoch 3/100
120/120 [==============================] - 6s 53ms/step - loss: 0.5527 - accuracy: 0.7499 - val_loss: 400.8427 - val_accuracy: 0.0245
Epoch 4/100
120/120 [==============================] - 6s 52ms/step - loss: 0.5307 - accuracy: 0.7563 - val_loss: 9.4444 - val_accuracy: 0.0271
Epoch 5/100
120/120 [==============================] - 7s 55ms/step - loss: 0.5172 - accuracy: 0.7587 - val_loss: 12.0613 - val_accuracy: 0.0501
Epoch 6/100
120/120 [==============================] - 6s 53ms/step - loss: 0.5254 - accuracy: 0.7543 - val_loss: 12.4562 - val_accuracy: 0.0443
Epoch 7/100
120/120 [==============================] - 7s 59ms/step - loss: 0.5375 - accuracy: 0.7581 - val_loss: 10.6288 - val_a

Epoch 57/100
120/120 [==============================] - 6s 52ms/step - loss: 0.4843 - accuracy: 0.7761 - val_loss: 15.2531 - val_accuracy: 0.0396
Epoch 58/100
120/120 [==============================] - 7s 55ms/step - loss: 0.4342 - accuracy: 0.7960 - val_loss: 21.6545 - val_accuracy: 0.0115
Epoch 59/100
120/120 [==============================] - 6s 53ms/step - loss: 0.4470 - accuracy: 0.7907 - val_loss: 12.0982 - val_accuracy: 0.0214
Epoch 60/100
120/120 [==============================] - 6s 50ms/step - loss: 0.4054 - accuracy: 0.8126 - val_loss: 12.8923 - val_accuracy: 0.0302
Epoch 61/100
120/120 [==============================] - 7s 58ms/step - loss: 0.3834 - accuracy: 0.8253 - val_loss: 12.7548 - val_accuracy: 0.0193
Epoch 62/100
120/120 [==============================] - 7s 60ms/step - loss: 0.3538 - accuracy: 0.8410 - val_loss: 13.8214 - val_accuracy: 0.0151
Epoch 63/100
120/120 [==============================] - 7s 60ms/step - loss: 0.3344 - accuracy: 0.8526 - val_loss: 15.1009 -

In [39]:
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, balanced_accuracy_score
batch_size = 64
# Evaluate the model on the test dataset
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test_categorical))
test_dataset = test_dataset.batch(batch_size)

test_loss, test_accuracy = new_model.evaluate(test_dataset)

# Make predictions on the test dataset
predictions = new_model.predict(test_dataset)

# Convert one-hot encoded predictions back to class labels
predicted_labels = np.argmax(predictions, axis=1)

# Calculate accuracy
accuracy = accuracy_score(Y_test, predicted_labels)

# Calculate F1 score
f1 = f1_score(Y_test, predicted_labels, average='weighted')

# Calculate balanced accuracy
balanced_acc = balanced_accuracy_score(Y_test, predicted_labels)

# Calculate confusion matrix
conf_matrix = confusion_matrix(Y_test, predicted_labels)

print(f"Test Loss: {test_loss}")
print(f"Test Accuracy: {test_accuracy}")
print(f"Accuracy: {accuracy}")
print(f"F1 Score: {f1}")
print(f"Balanced Accuracy: {balanced_acc}")
print("Cohen Kappa Score:")
from sklearn.metrics import cohen_kappa_score
k = cohen_kappa_score(Y_test, predicted_labels, weights='quadratic')
print(k)

27/27 [==============================] - 2s 27ms/step
Test Loss: 6.116485118865967
Test Accuracy: 0.5652431845664978
Accuracy: 0.5652431791221827
F1 Score: 0.5180683586315646
Balanced Accuracy: 0.327920549860205
Cohen Kappa Score:
0.21644955920001474
